# IBM Data Sciene

# Applied Data Science Capstone

## First Assignment

This notebook is used mainly to create the Capstone project, using Python and different Data Science related tools.

Notebook created by Carlos Granados, 2020

#### Part 1:

First assignment: call numpy and pandas...

In [1]:
# Import Libraries
import pandas as pd
import numpy as np

In [2]:
# Printing output...
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


#### Part 2:

Read the corresponding wikipedia article to import the neighborhoods name from Toronto with their respective borough names and Postal code. The data is a pandas Data Frame

1st step: Read the data from the URL and then store it in a string

In [3]:
# Data readed from the Wikipedia article...
import requests
import urllib.request
from lxml import html
from bs4 import BeautifulSoup
import re

# URL for wikipedia article with a list of all neighborhoods by district
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data_text = requests.get(wiki_url).content #text
#print(data_text)

# Web Page to string
raw_text = html.fromstring(data_text)

2nd step: Process the string, to extract only the valid names: those with a valid borough name and those that are not codes, since some additional tables with additional codes are imported from the Wikipedia article.

In [4]:
# To Extract Information

# 1st: get all poosible names, under the 'td' cathegory in the HTML
td_list = []
for s in raw_text.iter():
    if s.tag == 'td':
        # Save in the list all text (in special format)
        #  inside the td category
        td_list.append(s)
        #print(s.text_content())

# 2nd : Filter names
neighborhoods = []
i = 0
s = [0, 0, 0]
for td_e in td_list:
    # Convert each element in the list to a string
    text = str(td_e.text_content())
    # Save in a list all columns in the table (3 at the time)
    #  and ignorig the last two characters, all escape elements
    #  usually '\n'
    s[i] = text[0:-1]
    #print(i, s[i])
    i += 1
    # Make the inspection each 3 strings: We have a table with 3 Cols.
    if i == 3:
        # Save the neighborhood if the Borough is assigned
        # (and there is one) and the neighborhood is long enough
        # (some codes are readed, that not correspond to the table)
        if s[1] != 'Not assigned' and s[1] != '' and len(s[2]) > 3:
            #print(s)
            neighborhoods.append(s)
        if s[1] != 'Not assigned' and s[2] == 'Not assigned':
            s[2] = s[1]
            neighborhoods.append(s)
        s = [0, 0, 0]
        i = 0

# 3rd: Transform to a data frame, and rename the columns
cols = ['PostalCode', 'Borough', 'Neighborhood']
df_Toronto = pd.DataFrame(neighborhoods, columns=cols)
df_Toronto.sort_values('PostalCode').reset_index()
df_Toronto.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
df_Toronto.shape

(103, 3)

3rd step: Import geographical coordinates for each neighborhood, using geocoder library.

The geocoder library needs an `KEY_GOOGLE_API`, which for obvious reasons are not shown here

In [67]:
API_KEY_GOOGLE = ''

In [7]:
import geocoder

# Function to get the geographical coordinates for different postal 
#  codes in Toronto
def getCoords(postCode):
    ll_coords = None
    # Loop until get the coordinates
    i = 0
    while(ll_coords == None):
        i += 1
        address = '{}, Toronto'.format(postCode)
        g = geocoder.google(address, key=API_KEY_GOOGLE)
        ll_coords = g.latlng
    return ll_coords

Using the function `getCoords`, the loop will get each Postal Code from the data frame and will try to get ist geographical coordinates and storage them in the DF

In [8]:
# Loop for each post code, to get its coordinates

latAll = []
lonAll = []
for n in range(df_Toronto.shape[0]):
    postCode = df_Toronto['PostalCode'][n]
    ll_coords = getCoords(postCode)
    lat = ll_coords[0]
    lon = ll_coords[1]
    latAll.append(lat)
    lonAll.append(lon)
    print('Postal Code: {}, Coordinates: {}'.format(postCode, ll_coords))
    
df_Toronto['Latitude'] = latAll
df_Toronto['Longitude'] = lonAll

Postal Code: M3A, Coordinates: [43.7532586, -79.3296565]
Postal Code: M4A, Coordinates: [43.72588229999999, -79.3155716]
Postal Code: M5A, Coordinates: [43.6542599, -79.36063589999999]
Postal Code: M6A, Coordinates: [43.718518, -79.4647633]
Postal Code: M7A, Coordinates: [43.6623015, -79.3894938]
Postal Code: M9A, Coordinates: [43.6678556, -79.5322424]
Postal Code: M1B, Coordinates: [43.8066863, -79.1943534]
Postal Code: M3B, Coordinates: [43.7459058, -79.352188]
Postal Code: M4B, Coordinates: [43.7063972, -79.30993699999999]
Postal Code: M5B, Coordinates: [43.6571618, -79.3789371]
Postal Code: M6B, Coordinates: [43.709577, -79.4450726]
Postal Code: M9B, Coordinates: [43.65094320000001, -79.5547244]
Postal Code: M1C, Coordinates: [43.78453510000001, -79.1604971]
Postal Code: M3C, Coordinates: [43.7258997, -79.34092299999999]
Postal Code: M4C, Coordinates: [43.6953439, -79.3183887]
Postal Code: M5C, Coordinates: [43.6514939, -79.3754179]
Postal Code: M6C, Coordinates: [43.6937813, -79.4

In [16]:
df_Toronto.head(15)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [9]:
df_Toronto.shape

(103, 5)

4th step: Now use `sklearn` to use machine learning to classify the Toronto neighborhoods, based on its postal code. Follow closely the example for New York City (Manhattan)

In [68]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20182106' # Foursquare API version
LIMIT = 100

In [50]:
import json
from pandas.io.json import json_normalize

# Function to get the nearby venues for a set of locations

def getNearbyVenues(names, latAll, lonAll, radius=500):
    venue_list = []
    for name, lat, lon in zip(names, latAll, lonAll):
        #print(name)
        # Create the API and request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lon, 
            radius, 
            LIMIT)
        # Make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venue_list.append([(
            name, 
            lat, 
            lon, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        nearby_venues = pd.DataFrame([item for venue_list in venue_list for item in venue_list])
        nearby_venues.columns = ['PostalCode', 
                  'PC Latitude', 
                  'PC Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
        
    # Return
    return(nearby_venues)

And use the function to obtain the nearby venues...

As the postal code (inside Canada) of Toronto is "M", all the related data frames and list will be called `M_`

In [51]:
M_venues = getNearbyVenues(names=df_Toronto['PostalCode'],
                           latAll=df_Toronto['Latitude'],
                           lonAll=df_Toronto['Longitude'])

print('Nearby venues done')

M_venues.head(20)

Nearby venues done


,PostalCode,PC Latitude,PC Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
6,M4A,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
7,M4A,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
8,M5A,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
9,M5A,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop


Analysis of each zone (Post Code). The final data frame will be grouped by postal code and using the mean of the frequency of each category

In [52]:
#First, group by Postal Code
M_venues.groupby('PostalCode').count()

# One-hot enconding, and adding the Postal Code back to the Data Frame
M_oneHot = pd.get_dummies(M_venues[['Venue Category']], prefix="", prefix_sep="")
M_oneHot['PostalCode'] = M_venues['PostalCode']

# Move Postal Code column to the first column
fixed_columns = [M_oneHot.columns[-1]] + list(M_oneHot.columns[:-1])
M_oneHot = M_oneHot[fixed_columns]

M_oneHot.head(10)

,PostalCode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
M_group = M_oneHot.groupby('PostalCode').mean().reset_index()
M_group

,PostalCode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000
1,M1C,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000
2,M1E,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000
3,M1G,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000
4,M1H,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000
5,M1J,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000
6,M1K,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.125,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000
7,M1L,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000
8,M1M,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.500000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000
9,M1N,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000


In [54]:
# See all top 5 of venues for each Postal Code

numMax = 5
for pc in M_group['PostalCode']:
    print('-----'+pc+'-----')
    temp = M_group[M_group['PostalCode'] == pc].T.reset_index()
    temp.columns = ['Venue', 'Freq.']
    temp = temp.iloc[1:]
    temp['Freq.'] = temp['Freq.'].astype(float)
    temp = temp.round({'Freq.':2})
    print(temp.sort_values('Freq.', ascending=False).reset_index(drop=True).head(numMax))
    print('\n')


-----M1B-----
                      Venue  Freq.
0      Fast Food Restaurant    1.0
1             Movie Theater    0.0
2            Massage Studio    0.0
3            Medical Center    0.0
4  Mediterranean Restaurant    0.0


-----M1C-----
               Venue  Freq.
0     History Museum    0.5
1                Bar    0.5
2  Accessories Store    0.0
3  Mobile Phone Shop    0.0
4              Motel    0.0


-----M1E-----
                 Venue  Freq.
0         Intersection   0.14
1   Mexican Restaurant   0.14
2                 Bank   0.14
3  Rental Car Location   0.14
4    Electronics Store   0.14


-----M1G-----
                 Venue  Freq.
0          Coffee Shop   0.50
1         Soccer Field   0.25
2    Korean Restaurant   0.25
3   Miscellaneous Shop   0.00
4  Moroccan Restaurant   0.00


-----M1H-----
                 Venue  Freq.
0     Hakka Restaurant   0.12
1   Athletics & Sports   0.12
2  Fried Chicken Joint   0.12
3                 Bank   0.12
4               Bakery   0.12


--

                       Venue  Freq.
0             Clothing Store   0.09
1                Coffee Shop   0.08
2                       Café   0.04
3            Bubble Tea Shop   0.03
4  Middle Eastern Restaurant   0.03


-----M5C-----
                 Venue  Freq.
0          Coffee Shop   0.07
1                 Café   0.07
2         Cocktail Bar   0.04
3  American Restaurant   0.04
4            Gastropub   0.04


-----M5E-----
                Venue  Freq.
0         Coffee Shop   0.07
1        Cocktail Bar   0.05
2                Café   0.04
3  Seafood Restaurant   0.04
4            Beer Bar   0.04


-----M5G-----
                Venue  Freq.
0         Coffee Shop   0.18
1                Café   0.07
2  Italian Restaurant   0.07
3      Sandwich Place   0.05
4     Bubble Tea Shop   0.03


-----M5H-----
            Venue  Freq.
0     Coffee Shop   0.10
1            Café   0.05
2      Restaurant   0.04
3  Clothing Store   0.03
4             Gym   0.03


-----M5J-----
            Venue  Freq.
0

                  Venue  Freq.
0         Grocery Store   0.17
1              Pharmacy   0.08
2            Beer Store   0.08
3  Fast Food Restaurant   0.08
4   Fried Chicken Joint   0.08


-----M9W-----
                 Venue  Freq.
0  Rental Car Location    0.5
1            Drugstore    0.5
2    Accessories Store    0.0
3    Mobile Phone Shop    0.0
4                Motel    0.0




Now that the data is sorted, we can start with the clustering!

In [55]:
from sklearn.cluster import KMeans

# Number of Clusters. Lets start with 5, as in the NYC example
Kmax = 7

M_group_clustering = M_group.drop('PostalCode', 1)

# k-Means Clustering
kmM = KMeans(n_clusters=Kmax, random_state=0).fit(M_group_clustering)

# Check the cluster labels
kmM.labels_[0:10]

array([6, 5, 5, 3, 5, 4, 5, 5, 5, 5], dtype=int32)

Merge the cluster with the top 10 venues for each postal code.

It starts creating the top 10 data frame, and then merge it with the cluster

In [62]:
numTop = 10

indicators = ['st', 'nd', 'rd']

# Function to sort venues in descending order
def mostCommon(row, num_top):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top]

# Create columns according to number of top venues
columns = ['PostalCode']
for indx in np.arange(numTop):
    try:
        columns.append('{}{} Most Common Venue'.format(indx+1, indicators[indx]))
    except:
        columns.append('{}th Most Common Venue'.format(indx+1))

# Create a new dataframe
venuesSort = pd.DataFrame(columns=columns)
venuesSort['PostalCode'] = M_group['PostalCode']

for indx in np.arange(M_group.shape[0]):
    venuesSort.iloc[indx, 1:] = mostCommon(M_group.iloc[indx, :], numTop)
venuesSort.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Department Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant
1,M1C,Bar,History Museum,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
2,M1E,Rental Car Location,Electronics Store,Breakfast Spot,Medical Center,Intersection,Bank,Mexican Restaurant,Distribution Center,Diner,Discount Store
3,M1G,Coffee Shop,Soccer Field,Korean Restaurant,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Drugstore
4,M1H,Bakery,Hakka Restaurant,Bank,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Gas Station,Fried Chicken Joint,Discount Store,Dim Sum Restaurant


In [63]:
venuesSort.insert(0, 'Cluster Labels', kmM.labels_)

# Merge the original data frame with the cluster data and venues
M_merged = df_Toronto
M_merged = M_merged.join(venuesSort.set_index('PostalCode'), on='PostalCode')

# There are some NaNs in the clustering, maybe because there are
#  no data from the venues in such neighborhood!
M_merged.dropna(axis=0, inplace=True)

# For some reason, after the merging the Cluster labels are float...
M_merged['Cluster Labels'].astype('int32')

M_merged.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Fast Food Restaurant,Food & Drink Shop,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,3.0,Coffee Shop,Pizza Place,French Restaurant,Hockey Arena,Portuguese Restaurant,Yoga Studio,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,5.0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Restaurant,Theater,Café,Health Food Store,Historic Site
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,5.0,Women's Store,Furniture / Home Store,Clothing Store,Athletics & Sports,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Accessories Store,Vietnamese Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,5.0,Coffee Shop,Sushi Restaurant,Creperie,Bar,Beer Bar,Sandwich Place,Burrito Place,Restaurant,Café,Park
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,6.0,Fast Food Restaurant,Department Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant
7,M3B,North York,Don Mills,43.745906,-79.352188,5.0,Gym / Fitness Center,Caribbean Restaurant,Baseball Field,Japanese Restaurant,Café,Construction & Landscaping,Convenience Store,Event Space,Ethiopian Restaurant,Electronics Store
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,3.0,Pizza Place,Gym / Fitness Center,Gastropub,Pharmacy,Breakfast Spot,Bank,Intersection,Athletics & Sports,Fast Food Restaurant,Gay Bar
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,5.0,Clothing Store,Coffee Shop,Café,Japanese Restaurant,Middle Eastern Restaurant,Restaurant,Cosmetics Shop,Italian Restaurant,Bubble Tea Shop,Diner
10,M6B,North York,Glencairn,43.709577,-79.445073,1.0,Japanese Restaurant,Sushi Restaurant,Pizza Place,Pub,Yoga Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


In [64]:
M_merged.dtypes

PostalCode                 object
Borough                    object
Neighborhood               object
Latitude                  float64
Longitude                 float64
Cluster Labels            float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

The final step is to visualize it on a map!

In [66]:
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

Mlat = 43.741667
Mlon = -79.373333

# Constructor for the map
mapClus = folium.Map(location=[Mlat, Mlon],
                     zoom_start=10)

# Map colors
x = np.arange(numMax)
ys = [i + x + (i*x)**2 for i in range(Kmax)]
colorsArray = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colorsArray]

# Add markers to the map
markers_col = []
for lat, lon, pc, clus in zip(M_merged['Latitude'],
                              M_merged['Longitude'],
                              M_merged['PostalCode'],
                              M_merged['Cluster Labels']):
    label = folium.Popup(str(pc) + ' Cluster ' + str(clus), parse_html=True)
    j = int(clus)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[j-1],
        fill=True,
        fill_color=rainbow[j-1],
        fill_opacity=0.7    
    ).add_to(mapClus)

mapClus